<a href="https://colab.research.google.com/github/todnewman/coe_training/blob/master/Regression_for_NCAA_GamePlay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## NCAA Basketball Gameplay and Win Margin Predictor

This Notebook contains everything needed to predict the NCAA tournament.  The prediction is based upon over 10 years worth of NCAA tournament game results.  Features in the data consist of Kenpom.com features (I have to pay him $20 a year to update my data) and a few extras I have blended in.

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import matplotlib
import sklearn as sk
import numpy as np
from sklearn import preprocessing
from sklearn import metrics
import csv
from itertools import tee
import sys
from random import random
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


## Location of Data

Change the below to wherever the input files live (this .ipynb file and the actual_xx.csv files).  This is also where data outputs will go.

* Results_xx.csv:  These are the predictions for each round.
* pred_xx_out.csv: This is the predictive dataframe for each round (in case you're interested in looking the data over.
* NCAA_game_pred-xx-lasso.csv:  This is the results of the games played the number represents the win margin for the team in the first column.  The last two columns are the mean value of the win margin and the standard deviation.
* There is also a logfile, which should be very useful.  It will go wherever you tell it to go in the main block.

In [0]:
%ls /content

sample_data/


# Below is for use with the docker image

In [0]:
cd /host/data/sports

[Errno 2] No such file or directory: '/host/data/sports'
/content


# Dataset Test Area

Here's where I first import my dataset to get some ideas for future feature engineering.

In [0]:
!wget -O /content/kenpom_2019.csv 'https://raw.githubusercontent.com/todnewman/data/master/kenpom_2019.csv'
# Below is an old version of the historical dataset... new version is balanced and supports classification too
!wget -O /content/historical_tourney_data_2019.csv 'https://raw.githubusercontent.com/todnewman/data/f62d01432308decbb3b697a7bd27b7995efe1044/historical_tourney_data_2019.csv'
!wget -O /content/2019_NCAA_seed_data.csv 'https://raw.githubusercontent.com/todnewman/data/master/2019_NCAA_seed_data.csv'
!wget -O /content/actuals_2.csv 'https://raw.githubusercontent.com/todnewman/data/master/actuals_2.csv'
!wget -O /content/actuals_4.csv 'https://raw.githubusercontent.com/todnewman/data/master/actuals_4.csv'
!wget -O /content/actuals_8.csv 'https://raw.githubusercontent.com/todnewman/data/master/actuals_8.csv'
!wget -O /content/actuals_16.csv 'https://raw.githubusercontent.com/todnewman/data/master/actuals_16.csv'
!wget -O /content/actuals_32.csv 'https://raw.githubusercontent.com/todnewman/data/master/actuals_32.csv'
!wget -O /content/actuals_64.csv 'https://raw.githubusercontent.com/todnewman/data/master/actuals_64.csv'

%cd /content
%ls
out_filename = 'NCAA_games_2019.csv' # output file with predictions
filename_train = 'historical_tourney_data_2019.csv'
filename_predict = 'pred_test.csv'
df_t = pd.read_csv(filename_train, parse_dates=True)
#df_t.ix[:,40:57] # Print all records for the 17th through 27th column
print(df_t.columns)
df_t[:5]

--2019-03-18 20:31:46--  https://raw.githubusercontent.com/todnewman/data/master/kenpom_2019.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1993149 (1.9M) [text/plain]
Saving to: ‘/content/kenpom_2019.csv’

/content/kenpom_201 100%[===================>]   1.90M  --.-KB/s    in 0.07s   

2019-03-18 20:31:46 (25.9 MB/s) - ‘/content/kenpom_2019.csv’ saved [1993149/1993149]

--2019-03-18 20:31:48--  https://raw.githubusercontent.com/todnewman/data/master/historical_tourney_data_2019.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52903

,Season,TeamName,KeyField,CONF,SEED,WL%,EFGPct-OFF,TOPct-OFF,ORPct-OFF,FTRate-OFF,...,AdjTempo.L,Tempo-SUMM.L,OE.L,AdjOE.L,DE.L,AdjDE.L,AdjEM.L,TARGET16.L,Lscore,score_delta
0,2007,Niagara,2007Niagara,16,16,1.916667,52.2618,19.2451,32.1003,30.9432,...,72.62,71.49,101.70,102.81,101.30,94.62,8.19,0.0,69,8
1,2007,Boston College,2007Boston College,2,7,1.750000,49.0991,18.1444,37.0981,24.4745,...,67.02,65.04,97.49,93.25,106.07,105.23,-11.98,0.0,75,9
2,2007,Butler,2007Butler,8,5,4.142857,45.9375,21.5229,33.1892,42.3125,...,65.36,65.05,102.74,100.62,108.49,105.84,-5.22,0.0,46,11
3,2007,Georgetown,2007Georgetown,8,2,4.285714,44.0568,19.2612,33.9489,32.0930,...,68.87,66.47,107.42,108.33,97.34,88.61,19.72,0.0,55,25
4,2007,Indiana,2007Indiana,6,7,1.909091,46.2025,21.7668,31.7143,46.2025,...,64.98,62.34,102.44,100.52,98.73,93.95,6.57,0.0,57,13


In [0]:
%cd /content
%ls
df_temp = pd.read_csv('2019_NCAA_seed_data.csv')
df_temp

/content
2019_NCAA_seed_data.csv           NCAA_game_pred-16-lasso.csv  pred_4_out.csv
actuals_16.csv                    NCAA_game_pred-2-lasso.csv   pred_64_out.csv
actuals_2.csv                     NCAA_game_pred-32-lasso.csv  pred_8_out.csv
actuals_32.csv                    NCAA_game_pred-4-lasso.csv   results_16.csv
actuals_4.csv                     NCAA_game_pred-64-lasso.csv  results_2.csv
actuals_64.csv                    NCAA_game_pred-8-lasso.csv   results_32.csv
actuals_8.csv                     ncaa_logfile.txt             results_4.csv
drive/                            pred_16_out.csv              results_64.csv
historical_tourney_data_2019.csv  pred_2_out.csv               results_8.csv
kenpom_2019.csv                   pred_32_out.csv              sample_data/


,Regional,Seed,Team,Conference,Record,Berth Type,Overall Rank
0,1South,1,Virginia,NaN,NaN,NaN,NaN
1,1South,2,Tennessee,NaN,NaN,NaN,NaN
2,1South,3,Purdue,NaN,NaN,NaN,NaN
3,1South,4,Kansas St.,NaN,NaN,NaN,NaN
4,1South,5,Wisconsin,NaN,NaN,NaN,NaN
5,1South,6,Villanova,NaN,NaN,NaN,NaN
6,1South,7,Cincinnati,NaN,NaN,NaN,NaN
7,1South,8,Mississippi,NaN,NaN,NaN,NaN
8,1South,9,Oklahoma,NaN,NaN,NaN,NaN
9,1South,10,Iowa,NaN,NaN,NaN,NaN


# Prepare the dataframe - Feature Engineering

Here we have a few convenience functions that we call to get the Pandas dataframe ready for Machine Learning algorithms.  

prep_dataframe is intended to allow one spot for the developer to identify which features (columns) to bring in to the ML algorithm.

split_data takes a dataframe that has been prepared and splits it into training and test data sets.  This is also the location where I will do unbalanced dataset operations when this is necessary using the inbalanced learning algorithms

split_pred_data is a simple function to enable prediction on a predictive dataset

Mutual Information functions are for future use.

Correlation Matrix is used to identify the features with the most positive and negative correlations with the target.

In [0]:
'''Prep the dataset we wish to apply the model to.  There won't be a Target vector.'''
def prep_dataframe(df, flag, target, drop_seed):
    
    Name1 = []
    Name2 = []
    
    if flag == 'predict':        
        df[target] = 0 
        if drop_seed:
            df = df.drop(['Season', 'KeyField', 'SEED', 'SEED.L', 'TARGET16', 'Season.L',
                  'KeyField.L','TARGET16.L'], axis=1) # eliminate seed vector
        else:
            df = df.drop(['Season', 'KeyField', 'TARGET16', 'Season.L',
                  'KeyField.L','TARGET16.L'], axis=1) # eliminate seed vector


        Name1 = df.pop('TeamName')
        Name2 = df.pop('TeamName.L')
        Name1.append(Name1)
        Name2.append(Name2)
        return df, Name1, Name2
    else:
        
        if drop_seed:
            df = df.drop(['TeamName', 'TeamName.L', 'Season', 'KeyField', 'SEED', 'SEED.L', 'TARGET16', 'WScore','Season.L',
                  'KeyField.L','TARGET16.L','Lscore'], axis=1) # eliminate seed vector

        else:
            df = df.drop(['TeamName', 'TeamName.L', 'Season', 'KeyField', 'TARGET16', 'WScore','Season.L',
                  'KeyField.L','TARGET16.L','Lscore'], axis=1) # eliminate seed vector
            
        df = df.fillna(0)
        Name = 0
        
        return df, Name

In [0]:
def split_data(df_s, size, target, os_flag):
    ''' Split the data into training and testing sets.  Column headers below are custom for the "Douglas data set".
    NaN's are filled with zero to maintain consistent data shapes.
    '''
    from sklearn.model_selection import train_test_split
    from sklearn import preprocessing
    from imblearn.combine import SMOTEENN
    from imblearn.combine import SMOTETomek 
    from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler

    y = df_s.pop(target) # Remove the 'target' column that has the labels
    X = df_s.fillna(0)
    y = np.array(y).astype(int)
    X = np.array(X)
    
    # Scale the data so it works better with the ML algorithms... Also run oversampling algorithms
    
    
    if os_flag:
        sm = RandomOverSampler(random_state=41)    
        X, y = sm.fit_sample(X, y)
        
    if minmax_flag:
        X = preprocessing.MinMaxScaler().fit_transform(X)
    else:
        X = preprocessing.MaxAbsScaler().fit_transform(X)
    
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=size) # Keeping the test set small to evaluate timing

    return X_train, X_test, y_train, y_test

In [0]:
def split_pred_data(df_s, size, target):
    ''' Split the data into training and testing sets.  Column headers below are custom for the "Douglas data set".
    NaN's are filled with zero to maintain consistent data shapes.
    '''
    from sklearn.model_selection import train_test_split
    from sklearn import preprocessing
    df_s[target] = 0 # Just so we have a target vector in the DF, even though we won't use it

    y = df_s.pop(target) # Remove the 'target' column that has the labels
    X = df_s.fillna(0)
    y = np.array(y).astype(int)
    X = np.array(X)
    
    # Scale the data so it works better with the ML algorithms
    if minmax_flag:
        X = preprocessing.MinMaxScaler().fit_transform(X)
    else:
        X = preprocessing.MaxAbsScaler().fit_transform(X)
    
    return X, y

## Utility Functions below

* grab_kp_data - set up the actual predictive dataframe
* match_text - probably not used unless the team names don't match perfectly between datasets
* pairwise - small function to take two teams and place them adjacent

In [0]:
def grab_kp_data(df_winlose, pred_flag, year):
    '''
    Here we take the actual basketball data for the teams playing each other and build out the
    predictive dataframe.  I have a strange practice of flipping the teams so that each team is
    in the Team1 spot once.  This seems to make a difference.
    '''
    winner = pd.DataFrame()
    loser = pd.DataFrame()
    temp=pd.DataFrame()
    temp1 = pd.DataFrame()
    temp2 = pd.DataFrame()
    combined = pd.DataFrame()
    combined2 = pd.DataFrame()
    
    filename_kp_data = 'kenpom_2019.csv'
    similarity_ratio = .93
    
    ncaa_data = pd.read_csv(filename_kp_data, parse_dates=True)
    
    if pred_flag:
        ncaa_data = ncaa_data[ncaa_data['Season']==year]
        
    for i, row in df_winlose.iterrows():
        key_win =  ("%s%s" % (row['Season'], row['WTeamID']))
        key_lose = ("%s%s" % (row['Season'], row['LTeamID']))
        
        #
        # Use the below if you're having text dis-similarity issues and the keys aren't equal
        #       
        #win_team_filter = match_text(ncaa_data['KeyField'], key_win, similarity_ratio) # This is the KeyField
        #lose_team_filter = match_text(ncaa_data['KeyField'], key_lose, similarity_ratio) # THis is the KeyField
        
        win_team_filter = ncaa_data['KeyField']==(key_win) # This is the KeyField
        lose_team_filter = ncaa_data['KeyField']==(key_lose) # THis is the KeyField
        
        temp2=ncaa_data[lose_team_filter]
        temp1=ncaa_data[win_team_filter]
        
        if not pred_flag:
            temp1['WScore'] = row['WScore']
            temp2['LScore'] = row['LScore']
        
        temp2.index = temp1.index.copy()
        temp2 = temp2.add_suffix(".L")
        winner = pd.concat([temp1, temp2], axis=1)
        
        if not pred_flag:
            winner['score_delta'] = winner.WScore - winner.LScore
            loser = pd.concat([temp2, temp1], axis=1)
            loser['score_delta'] = loser.WScore - loser.LScore
        
        
        combined = combined.append(winner)
        
        if not pred_flag:
            combined2 = combined2.append(loser)
    
    if not pred_flag:
        return (combined, combined2)
    else:
        return (combined)
        
def match_text(df1, pred, ratio):
    from difflib import SequenceMatcher
    filter = []
    for name in df1:
        m = SequenceMatcher(None, name, pred)
        # Use the below if you're having trouble matching team names...
        #if m.ratio() > .9:
        #    print(name, pred, m.ratio())
        if m.ratio() > ratio:
            filter.append(True)
        else:
            filter.append(False)
    return filter
                
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)
                    


##  Mutual Information Functions

These functions are here in case I find a way to use them to capture mutual information between each feature and the target.  Right now, the best way I have found is a basic correlation matrix.

* Currently We're not using the kernel_pmi_func, pmi_func, and mutual_info functions.  Kept them here because it took some work to figure out how to code them and maybe they'll be useful someday.  
* Essentially what we ARE using is the SKLEARN mutual_info_clasif method (inside the get_mutual_information function.

In [0]:
def kernel_pmi_func(df, x, y):
    from sklearn.neighbors.kde import KernelDensity

    # reshape data
    x = np.array(df[x])
    y = np.array(df[y])
    x_y = np.stack((x, y), axis=-1)

    # kernel density estimation
    kde_x = KernelDensity(kernel='gaussian', bandwidth=0.1).fit(x[:, np.newaxis])
    kde_y = KernelDensity(kernel='gaussian', bandwidth=0.1).fit(y[:, np.newaxis])
    kde_x_y = KernelDensity(kernel='gaussian', bandwidth=0.1).fit(x_y)

    # score
    p_x = pd.Series(np.exp(kde_x.score_samples(x[:, np.newaxis])))
    p_y = pd.Series(np.exp(kde_y.score_samples(y[:, np.newaxis])))
    p_x_y = pd.Series(np.exp(kde_x_y.score_samples(x_y)))   

    df['pmi'] = np.log( p_x_y / (p_x * p_y) )
    
    return df

def pmi_func(df, x, y):
    freq_x = df.groupby(x)[x].transform('count')
    freq_y = df.groupby(y)[y].transform('count')
    freq_x_y = df.groupby([x, y])[x].transform('count')
    df['pmi'] = np.log( len(df.index) *  (freq_x_y / (freq_x * freq_y)) )
    return df['pmi']

def mutual_info(p):
    p_x=p.sum(axis=1)
    p_y=p.sum(axis=0)
    I=0.0
    for i_y in p.index:
        for i_x in p.columns:
            #if ((p_x[i_y]*p[i_x]).any() > 0):
            I+=(p.ix[i_y,i_x]*np.log2(p.ix[i_y,i_x]/(p_x[i_y]*p[i_x]))).values[0]
    return I

from sklearn.feature_selection import mutual_info_classif
def get_mutual_information(df, target, range):
    # Split data:
    df_copy = pd.DataFrame()
    df_copy = df
    
    y = df_copy.pop(target)
    X = df_copy.fillna(0)
    X = np.array(X)

    df_n = pd.DataFrame()
    df_n['MI'] = mutual_info_classif(X, y)
    df_n['Keys'] = df_copy.keys()
    df_n = df_n.sort_values(['MI'], ascending=False)
    features = np.array(df_n['Keys'][1:range].tolist())
    
    return features

## Feature Selection

This module is designed to select the most informative features.  Right now it's using a basic correlation matrix and then selecting the most positive and negative correlated features using a variable called "half_range".  Setting half_range to 4 will take the top 25% of the ordered matrix as well as the bottom 25%.

In [0]:
def feature_select(target, df_p, mi_flag): 
    
    half_range = 3.0  # This will determine how many of the most positive and negative correlated features are used
       
    # Open a dataframe just to create the correlation matrix                             
    df_new = pd.DataFrame()
    df_copy = pd.DataFrame()
    df_copy['Target'] = df_p[target]
    
    # Convert Objects to Values
    cols = df_p.columns
    df_p[cols] = df_p[cols].apply(pd.to_numeric, errors='coerce')
    
    df_new = df_p.corr().sort_values(target, ascending = False) 

    size = len(df_new)
    
    ratio = (1/half_range) # will let us pick the highest and lowest correlated features by this ratio
    
    upper_limit = int(size*ratio)
    lower_limit = int(size*(1-ratio))
    upper_range = df_new[target].index[1:upper_limit].tolist()
    lower_range = df_new[target].index[lower_limit:size].tolist()
    
    '''
    Below allows us to switch between mutual information-based and correlation-based feature selectors
    '''
    if mi_flag:
        total_range = get_mutual_information(df_p, target, upper_limit)
    else:
        total_range = (upper_range + lower_range)
        
    target_vector = df_copy['Target']
    df_p = df_p[total_range] # limit the dataframe used for training to just the most informative features
    df_p['Target'] = target_vector


    return (df_p, 'Target', total_range)

## Process data from each round

This takes the results from the previous round and organizes the data for the next round.  It creates the file to be used for inference by the model in the next round.

The function that handles the results values and calls process data is included in this block too.

In [0]:
def process_data(results, seeds, round_t, verbose):
    '''
    Pass in results of the prediction and output the file used for prediction.
    '''
    
    curr_year = 2019 # change every year
    
    # Dataframes that we'll use
    df_prediction_flip = pd.DataFrame()
    df_prediction = pd.DataFrame()
    
    # Lists that we'll use
    w_team = []
    l_team = []
    sorted_t = []
    
    # Constants
    TEAM_REGION=0
    TEAM_SEED=1
   
    # Output Filename
    pred_file = ('pred_%s_out.csv' % round_t)

    #
    #  Dictionaries below that let us line up the teams playing each other - slightly different for each
    #  round of the bracket.
    #
    if round_t == 64:
        bracket_dict = { 1:0, 16:1, 8:2, 9:3, 5:4, 12:5, 4:6, 13:7, 6:8, 11:9, 3:10, 14:11, 7:12, 10:13, 2:14, 15:15}
    elif round_t == 32:
        bracket_dict = { 1:0, 16:0, 8:1, 9:1, 5:2, 12:2, 4:3, 13:3, 6:4, 11:4, 3:5, 14:5, 7:6, 10:6, 2:7,15:7}
    elif round_t == 16:
        bracket_dict = {1:0, 16:0, 8:0, 9:0, 5:1, 12:1, 4:1, 13:1, 6:2, 11:2, 3:2, 14:2, 7:3, 10:3, 2:3,15:3}
    elif round_t == 8:
        bracket_dict = {1:0, 16:0, 8:0, 9:0, 5:0, 12:0, 4:0, 13:0, 6:1, 11:1, 3:1, 14:1, 7:1, 10:1, 2:1,15:1}
    else:
        bracket_dict = {1:0, 16:0, 8:0, 9:0, 5:0, 12:0, 4:0, 13:0, 6:0, 11:0, 3:0, 14:0, 7:0, 10:0, 2:0,15:0}

    try:
        results = results.drop('Unnamed: 0', axis=1)
    except:
        pass

    #
    # Lets find the teams from the NCAA database that are in our results file.
    # This will filter all our winners.
    #
    if round_t != 64:
        filter_winners = seeds['Team'].isin(results['res'])

        seeds = seeds[filter_winners] # now the seeds dataframe is just the teams playing in this round.

    #
    # Sort the DataFrame so teams playing each other will be adjacent rows
    #
    for i in seeds['Seed']:
        sorted_t.append(bracket_dict[i]) 
    try:
        seeds = seeds.drop('Unnamed: 0', axis=1)
    except:
        pass
    
    seeds['Sort_by'] = sorted_t
    
    #
    # Temporary DataFrame to sort by Regions first and then by Seed Values to get opponents in adjacent rows.
    #
    df_temp = (seeds.sort_values(['Regional','Sort_by'], ascending = False))
    
    df_temp = df_temp.reset_index() # Reset index to allow pairwise to work
    del df_temp['index']
    
    if verbose:
        print("This is the sorted dataframe by seed value.  Odd rows should be playing even rows below them.\n")
        print(df_temp)

    #
    # Here we take the bracket data and grab both team names into a list.
    #
    for (i1, row1), (i2, row2) in pairwise(df_temp.iterrows()):
        if i1%2 == 0: # Every second row
            w_team.append(row1[2])
            l_team.append(row2[2])
    #
    # Create the DataFrame with the info we need to be able to 
    df_prediction['WTeamID'] = w_team
    df_prediction['LTeamID'] = l_team
    df_prediction_flip['WTeamID'] = l_team
    df_prediction_flip['LTeamID'] = w_team
    df_prediction = pd.concat([df_prediction, df_prediction_flip], axis=0)
    df_prediction['Season'] = curr_year

    if verbose:
        print("This is the dataframe we end up routing to the play_games function.")
        print("It should be ordered such that the teams playing each other are adjacent.\n")
        print(df_prediction)
        
    pred_flag = True
    winnerp = grab_kp_data(df_prediction, pred_flag, curr_year)
    winnerp.to_csv(pred_file)
    return winnerp



# REGRESSION Function

The idea of this function is that it can predict the margin of victory for teams that are matched up in the tournament.  A positive margin of victory reflects a predicted victory for the team to the left.  A negative margin of victory reflects a predicted win for the team to the right.

In [0]:
def run_regression(df, df_predict, model, round_t):
    # df is the training data from many years of NCAA tourney games
    # df_predict is the list of ordered contestants plus their current year stats
    
    target = 'score_delta'

    best_features = []

    df_test_train = df
    
    # Define the technique to fit the model the data.  LassoCV has lower R^2 scores but seems more consistent
    # in earlier rounds.
    
    if (round_t in [64, 32, 16, 8]):
        os_flag = False
        drop_seed = True
        model = ElasticNetCV(cv=5, alphas=[ 1e-2, 1e-1, 1], fit_intercept=False)
    else:
        os_flag = True
        drop_seed = False
        #model = MLPRegressor(hidden_layer_sizes=(64,32,8), alpha=.001, early_stopping=True)
        model = MLPRegressor(hidden_layer_sizes=(16,24,8), alpha=.001, early_stopping=True)


    '''
    First, process the training data set.
    '''
    
    df_train, Name = prep_dataframe(df_test_train, 'train', target, drop_seed)
    df_train, target, best_features = feature_select(target, df_train, mi_flag)
    

    # Split data: 20% of data goes into the test set
    X_train, X_test, y_train, y_test = split_data(df_train, 0.20, target, os_flag)
        
    #print(model)
        
    model.fit(X_train, y_train)
    
    expected = y_test # This is the values of the Target that we 'know' from the historical data
    predicted = model.predict(X_test) # These are the Targets we predict (to compare to the expected values later)
    
    
    r2_score_test = r2_score(expected, predicted)
    
    '''
    Now we use the trained model to predict how this seasons games will go.
    '''
        
    # Feature Selection using 'best_features' from the run_corr feature selection call.  Done to align
    #    Prediction data with the Training data.
    
    df_pred, Name1, Name2 = prep_dataframe(df_predict, 'predict', target, drop_seed)
    df_pred = df_pred[best_features]
    X_predict, y_bogus = split_pred_data(df_pred, 0, target)  # Do scaling of data to match training data
    predict_val = model.predict(X_predict)   
    
    return (Name1, Name2, predict_val, model, r2_score_test)



## Handle the Results of the Played Games

play_games: sets up the data to run the regression function.

handle_results: Takes the combined dataframe from all of the played games and routes to the determine_win function to provide a list of teams that will go forward to the next round (res_df)

determine_win: takes the combined post-game dataframe and evaluates who to declare the winner through a basic set of rules.  Returns this list of teams to the calling function.

compare_scores: conduct a two-factor T-Confidence in Means test to account for teams with extreme variability in scores.

In [0]:
def play_games(pred, round_t, df_train, verbose ):
    '''
    This is the executive for the gameplaying feature.
    '''
    r2 = []
    
    n_iter = 15  # Number of Games to run
    combined = pd.DataFrame()
    first_time = True
    
    filename_predict = ('pred_%s_out.csv' % round_t)
    if rf_flag:
        model = 'rf'
    else:
        model = 'lasso'

    for n in range (0, n_iter):
        Team1, Team2, Prediction, modelinfo, r2_score = run_regression(df_train, pred, model, round_t)
        if first_time:
            combined['Team1'] = Team1.values
            combined['Team2'] = Team2.values
        first_time = False
        combined[n] = Prediction
        r2.append(r2_score)   
    print("The Average r2 score for these %d iterations is %7.2f" % (n_iter, np.mean(r2)))
    
    combined['mean'] = combined.mean(axis = 1)
    combined['std'] = combined.std(axis = 1)
    combined.sort_values(['mean'], ascending = False).to_csv('NCAA_game_pred-%s-lasso.csv' % round_t)
    
    if verbose:
        print ("Here are the results from regression from the play_games function for %s iterations" % n_iter)
        print (combined[['Team1', 'Team2', 'mean', 'std']].sort_values(['mean'], ascending = False))
    return combined, modelinfo

def handle_results(combined, round_t, verbose):
    filename_actuals = '/content/actuals_%s.csv' % round_t
    df_actuals = pd.read_csv(filename_actuals, header=None, parse_dates=True)
    
    results = determine_win(combined, round_t, verbose)
    res_df = pd.DataFrame()
    temporary_df = pd.DataFrame()
    res_df['res'] = results
    
    res_list = list(set(res_df['res']))
    actual_list = list(set(df_actuals[0]))
    #hits = df_actuals.isin(res_list)
    try:
        hits = res_df.isin(actual_list)
    except:
        hits = 0
    print("Correctly Selected: ", res_df[hits])
    #logger.debug("Correctly Selected: %s " % res_df[hits])
    logger.debug("Correctly Selected: %s " % res_df)
    temporary_df['results'] = res_list

    temporary_df['correct'] = hits
    #temporary_df['actuals'] = actual_list
    hit_count = temporary_df['correct'].sum()
    score = 10*(hit_count)*64.0/round_t # Matches ESPN Scoring

    res_df.to_csv('results_%s.csv' % round_t)
    print("# Correct for round %s:  %s" % (round_t, hit_count))
    print("score for round %s:  %s" % (round_t, score))
    logger.debug("# Correct for round %s:  %s" % (round_t, hit_count))
    logger.debug("score for round %s:  %s" % (round_t, score))
    #print("Correct Predictions\n", temporary_df)
    return res_df, score


def determine_win(df, round_t, verbose):
    seed1 = []
    seed2 = []
    winval2 = []
    std = []
    teams = []
    
    scaled_value_needed_for_underdog_win = .05
    scaled_value_needed_for_lowseed_win = 0
    
    filename_predict = ('pred_%s_out.csv' % round_t)
   
    
    df_p = pd.read_csv(filename_predict, parse_dates=True)
    seed_dict = df_p.set_index('TeamName').to_dict()['SEED']
    
    df['win_val'] = df['mean']
    
    #adj_mean = (df['mean'].mean())
    #df['adj_mean'] = df['mean'] - adj_mean
    #df['win_val'] = df['mean']
    
    win_val_dict = df.set_index('Team1').to_dict()['win_val']
    std_dict = df.set_index('Team1').to_dict()['std']
    
    df = df.sort_values(['win_val'], ascending = False)
        
    for i, row in df.iterrows(): 
        seed1.append(seed_dict[row['Team1']])
        seed2.append(seed_dict[row['Team2']])
        winval2.append(win_val_dict[row['Team2']])
        std.append(std_dict[row['Team2']])
        
    df['seed1'] = seed1 
    df['seed2'] = seed2 
    df['win_val2'] = winval2
    df['std2'] = std
    
    if verbose:
        print("Winning Team  Losing Team  Win Value1  Win Value2")
        print(df[['Team1', 'Team2', 'win_val', 'win_val2']][0:35])
        #print(df[['Team1', 'Team2', 'win_val', 'win_val2']][30:63])
        
    #    
    # Apply rules to determine who to predict as the winner.  Remember that the records are passed in the dataframe
    #   ranked by the win_val column.  
    #
    
    for i, team in enumerate(df['Team1'].values):
        team2 = df['Team2'].iloc[i]
        team_score = df['win_val'].fillna(0).iloc[i]
        team2_score = df['win_val2'].fillna(0).iloc[i]
        team_seed = df['seed1'].fillna(0).iloc[i]
        team2_seed = df['seed2'].fillna(0).iloc[i]
        team_std = df['std'].fillna(0).iloc[i]
        team2_std = df['std2'].fillna(0).iloc[i]
        
        conf_int_ul, conf_int_ll = compare_scores(team_score, team_std, team2_score, team2_std)
        conf_value = 10*(conf_int_ul-conf_int_ll)
        
        logger.debug("%s %d %7.2f ** %s %d %7.2f --- %7.2f\n" % 
                     (team, team_seed, team_score, team2, team2_seed, team2_score, conf_value))
        
        #
        # Rule One:  Top seeds have to be beaten by more than the magic number to "lose"
        #
        if team not in teams and team2 not in teams:
            if team_seed < team2_seed:  # This is a higher seed
            # The opponent's win value is not greater than the magic number
                if (team_score - team2_score) > 0:
                    teams.append(team)
                elif (team2_score - team_score) > -conf_value:
                    teams.append(team2)
                    
                else: teams.append(team)
            elif team_seed == team2_seed:
                if team_score > team2_score:
                    teams.append(team)
                else: teams.append(team2)
            
            
    
    if verbose:
        print(" * Here are the Teams Going into the next round %s" % round_t)
        for i, t in enumerate(teams):
            print("%s) %s" % (i, t))

    return teams  

def compare_scores(mean1, std1, mean2, std2):
    '''
    Here we want to calculate the confidence interval for the difference in means.  This is
    necessary due to the extreme standard deviations in predicted scores that some teams generate.
    The theory is that this will account for teams with good numbers that have some latent 
    inconsistency.
    '''
    
    import numpy as np, statsmodels.stats.api as sms

    X1 = np.random.normal(mean1, std1, 5000)
    X2 = np.random.normal(mean2, std2, 5000)

    cm = sms.CompareMeans(sms.DescrStatsW(X1), sms.DescrStatsW(X2))
    return (cm.tconfint_diff(usevar='unequal'))




# MAIN function



In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression, BayesianRidge, LassoCV, RidgeCV, LassoLarsCV, MultiTaskLassoCV, ElasticNetCV
from sklearn.svm import SVR
from sklearn.model_selection import KFold
import logging
import warnings
warnings.filterwarnings("ignore")

# Input Files
filename_train = 'historical_tourney_data_2019.csv' # This is the historical dataset from years of playoffs results
ncaa_file = '2019_NCAA_seed_data.csv' # This year I got this data from Wikipedia.  May be a better source...

# Logging setup
#logname = '/home/tod/Documents/sports/ncaa_logfile.txt'
logname = '/content/ncaa_logfile.txt'

logger = logging.getLogger('NCAA Predictor')
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler(logname)
fh.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
logger.addHandler(fh)

logger.info("NCAA tourney output RUN Starts here")

# Global Variables
global mi_flag # Use mutual information instead of correlation.  
global os_flag # Use random oversampling to address imbalance in the target
global minmax_flag # Use the MinMax Scaler instead of MaxAbs
global rf_flag # Use the elastic lasso or rf(mlp).  If this is True, os_flag should be set to True too.

mi_flag = False
os_flag = False
minmax_flag = True
rf_flag = False 
predict_only = False # don't score the results

logger.info("mi_flag %s * os_flag %s * minmax_flag %s * rf_flag %s\n" % 
             ( mi_flag,  os_flag, minmax_flag,  rf_flag))

curr_year = 2019 # change every year
n_iter = 1 # of times we want to loop
overall_score = []

# Iterate through 'n_iter' iterations.  Simplest thing may be to set n_iter to 1, but here it is just in case.
for n in range(0,n_iter):
    print("******************** ITERATION %d *************************" % n)
    df_train = pd.read_csv(filename_train)
    ncaa_data = pd.read_csv(ncaa_file)
    
    res_df = 0
    score_list = []


    for round_t in [64, 32, 16, 8, 4, 2]:
        print("running in round: %s" % round_t)
        verbose = True
        pred_df = process_data(res_df, ncaa_data, round_t, verbose)
        combined, model = play_games(pred_df, round_t, df_train, verbose)
    
        logger.info("Model Info: %s" % model)
        verbose = False
        res_df, score = handle_results(combined, round_t, verbose)
        if not predict_only:
            
            score_list.append(score)
    
    if not predict_only:
        print ("Overall Score is:", sum(score_list))
        logger.info("Overall Score is: %d" % sum(score_list))
        logger.info(score_list)
        logger.info("********************************************************************************************************")
        logger.info(" ")
        print(score_list)
        overall_score.append(sum(score_list))
        print("Overall scores for %d runs" % n_iter)
        print(overall_score)


******************** ITERATION 0 *************************
running in round: 64
This is the sorted dataframe by seed value.  Odd rows should be playing even rows below them.

    Regional  Seed                 Team  Conference  Record  Berth Type  \
0       West    15              Montana         NaN     NaN         NaN   
1       West     2             Michigan         NaN     NaN         NaN   
2       West    10              Florida         NaN     NaN         NaN   
3       West     7               Nevada         NaN     NaN         NaN   
4       West    14    Northern Kentucky         NaN     NaN         NaN   
5       West     3           Texas Tech         NaN     NaN         NaN   
6       West    11           St. John's         NaN     NaN         NaN   
7       West     6              Buffalo         NaN     NaN         NaN   
8       West    13              Vermont         NaN     NaN         NaN   
9       West     4          Florida St.         NaN     NaN         NaN   


In [0]:
#Here's where you can mount your own Google Drive and copy results from /content into it...

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'My Drive'/data
%ls




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/data
1_FBB_2017_matrix.png              PlayByPlay_2011.zip
1_rain_confusion_matrix.png        PlayByPlay_2012.zip
2_FBB_2017_matrix.png              PlayByPlay_2013.zip
2_rain_confusion_matrix.png        PlayByPlay_2014.zip
Cities.csv                         PlayByPlay_2015.zip
Conferences.csv                    PlayByPlay_2016.zip
ConferenceTourneyGames.csv         PlayByPlay_2017.zip
corr_mat.png                       PlayByPlay_2018.zip
DataFiles.zip                      Players_2010.csv
Events_2010.csv                    Players_2011.csv
Events_2011.csv                    Players_2012.csv
Events_2012.csv                    Players_2013.csv
Events_2013.csv                    Players_2014.csv
Events_2014.csv                    Players_2015.csv
Events_2015.csv                    Players_2016.csv
Events_2016.csv                    Pla

In [0]:
%ls /content


2019_NCAA_seed_data.csv           NCAA_game_pred-16-lasso.csv  pred_4_out.csv
actuals_16.csv                    NCAA_game_pred-2-lasso.csv   pred_64_out.csv
actuals_2.csv                     NCAA_game_pred-32-lasso.csv  pred_8_out.csv
actuals_32.csv                    NCAA_game_pred-4-lasso.csv   results_16.csv
actuals_4.csv                     NCAA_game_pred-64-lasso.csv  results_2.csv
actuals_64.csv                    NCAA_game_pred-8-lasso.csv   results_32.csv
actuals_8.csv                     ncaa_logfile.txt             results_4.csv
drive/                            pred_16_out.csv              results_64.csv
historical_tourney_data_2019.csv  pred_2_out.csv               results_8.csv
kenpom_2019.csv                   pred_32_out.csv              sample_data/


In [0]:
# Copies results to the google drive

%cp /content/results*csv .